Setup conda environment

In [ ]:
%env PYTHONPATH = # /env/python
!wget https://repo.anaconda.com/miniconda/Miniconda3-py38_4.12.0-Linux-x86_64.sh
!chmod +x Miniconda3-py38_4.12.0-Linux-x86_64.sh
!./Miniconda3-py38_4.12.0-Linux-x86_64.sh -b -f -p /usr/local
!conda update conda -y
import sys
sys.path.append('/usr/local/lib/python3.8/site-packages')
!conda create -n myenv python=3.7 -y

In [ ]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv
pip install numpy
pip install numba==0.53.1
pip install protobuf==3.20.0
pip install onnx==1.9.0
pip install onnxruntime==1.9.0
pip install onnxoptimizer==0.2.6
pip install sympy
pip install pillow
pip install tensorflow
pip install tf2onnx
pip install scikit-learn

Mount google drive and download esp-dl source code

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!git clone --recursive --depth=1 https://github.com/espressif/esp-dl.git /content/drive/MyDrive/hand_recognition/esp-dl

Prepare data

In [ ]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv
python3 /content/drive/MyDrive/hand_recognition/prepare_data.py

Train model

In [ ]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv
python3 /content/drive/MyDrive/hand_recognition/train.py

Convert model to onnx format

In [ ]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv
wd='/content/drive/MyDrive/hand_recognition/trained_model'
python -m tf2onnx.convert --saved-model $wd/tmp_model --output $wd/hand_rec_model.onnx

Use esp-dl quantization tool to quantize and generate .cpp/.hpp files for deployment

In [ ]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv
wd='/content/drive/MyDrive/hand_recognition/'
python /$wd/esp_dl_quant.py -e $wd/esp-dl -m $wd/trained_model/hand_rec_model.onnx -t esp32 -x $wd/data/X_cal.pkl -y $wd/data/y_cal.pkl -o $wd/trained_model/